# 1 Mechine Learning

In [56]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [39]:
df = pd.read_csv("data/titanic.csv", index_col="PassengerId")
df.drop(columns=["Name","Ticket","Age","Cabin"],inplace=True)

In [57]:
# X = input dan y = Target

X = df.drop(columns=["Survived"])
y = df["Survived"]

#split data                  
#stratify berguna untuk mengklasifikasikan berdasarkan label survive dan not survive
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 6), (179, 6), (712,), (179,))

## Prepocessor Menggunakan Pipe line Agar data tidak bocor / data leakage

In [58]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

In [59]:
numerical_pipeline = Pipeline([
    #nama steps 
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", MinMaxScaler())
])

categorical_pipeline = Pipeline([ 
    ("imputer", SimpleImputer(strategy="most_frequent")),
    #encode
    ("onehot", OneHotEncoder())
])

In [60]:
#berguna untuk memfilter kolom mana yang masuk pipa numerik dan pipa kategori
from sklearn.compose import ColumnTransformer

In [61]:
X_train.head()

,Pclass,Sex,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,
693,3,male,0,0,56.4958,S
482,2,male,0,0,0.0000,S
528,1,male,0,0,221.7792,S
856,3,female,0,1,9.3500,S
802,2,female,1,1,26.2500,S


In [62]:
preprocesor = ColumnTransformer([
    ("numeric", numerical_pipeline, ["SibSp","SibSp","Fare"]),
    ("categoric",categorical_pipeline,["Pclass","Sex","Embarked"])
])

### Pipeline

In [63]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline([
    ("prep", preprocesor),
    ("algo",KNeighborsClassifier())
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['SibSp', 'SibSp', 'Fare']),
                                                 ('categoric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Pclass', 'Sex',
                           

In [64]:
pipeline.score(X_test, y_test)

0.770949720670391

# 2 Grid Search CV(Cross Validation) ---Final---

In [65]:
from sklearn.model_selection import GridSearchCV

#paremeter tuning
parameter = {
    #kenapa ada algo karna untuk memberi tahu n_neigborh punya nya pipeline KNeighborsClassifier()
    "algo__n_neighbors":range(1,51,2),
    "algo__weights":["uniform","distance"],
    "algo__p":[1,2]
}

#biasany crosvalidation 3-5
#n_jobs mw berapa banyak core cpu di pakai
model = GridSearchCV(pipeline, parameter,cv=3,n_jobs=-1,verbose =1)
model.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          MinMaxScaler())]),
                                                                         ['SibSp',
                                                                          'SibSp',
                                                                          'Fare']),
                                                                        ('categoric',
                                                                         Pipeline(steps=[('impu

In [69]:
pd.DataFrame(model.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algo__n_neighbors,param_algo__p,param_algo__weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
26,0.024777,0.005470,0.031737,0.001040,13,2,uniform,"{'algo__n_neighbors': 13, 'algo__p': 2, 'algo_...",0.819328,0.805907,0.805907,0.810381,0.006327,1
24,0.023580,0.001340,0.036673,0.000629,13,1,uniform,"{'algo__n_neighbors': 13, 'algo__p': 1, 'algo_...",0.819328,0.805907,0.805907,0.810381,0.006327,1
44,0.020667,0.000804,0.031164,0.000158,23,1,uniform,"{'algo__n_neighbors': 23, 'algo__p': 1, 'algo_...",0.810924,0.818565,0.797468,0.808986,0.008721,3
46,0.022306,0.001619,0.037102,0.006838,23,2,uniform,"{'algo__n_neighbors': 23, 'algo__p': 2, 'algo_...",0.810924,0.818565,0.797468,0.808986,0.008721,3
36,0.019805,0.000161,0.031658,0.000597,19,1,uniform,"{'algo__n_neighbors': 19, 'algo__p': 1, 'algo_...",0.798319,0.814346,0.810127,0.807597,0.006783,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,0.020887,0.000518,0.013277,0.000092,3,2,distance,"{'algo__n_neighbors': 3, 'algo__p': 2, 'algo__...",0.768908,0.742616,0.763713,0.758412,0.011369,96
2,0.027024,0.001726,0.031724,0.001413,1,2,uniform,"{'algo__n_neighbors': 1, 'algo__p': 2, 'algo__...",0.777311,0.700422,0.729958,0.735897,0.031669,97
1,0.025005,0.001352,0.015616,0.001585,1,1,distance,"{'algo__n_neighbors': 1, 'algo__p': 1, 'algo__...",0.777311,0.700422,0.729958,0.735897,0.031669,97
3,0.022359,0.001594,0.014758,0.001027,1,2,distance,"{'algo__n_neighbors': 1, 'algo__p': 2, 'algo__...",0.777311,0.700422,0.729958,0.735897,0.031669,97


In [66]:
model.best_params_

{'algo__n_neighbors': 13, 'algo__p': 1, 'algo__weights': 'uniform'}

In [70]:
#hasil akhir yang di peroleh
#tidak ada data lekage semua aman dengan menggunakan GridSearchCV
model.score(X_train, y_train),  model.score(X_test, y_test)

(0.8258426966292135, 0.776536312849162)